In [1]:

import numpy as np
import pandas as pd

class MonteCarloSimulator:
    """
    Generic Monte Carlo simulator.

    Parameters
    ----------
    n_sims : int
        Number of simulation paths (trials).
    horizon : int
        Number of time steps per simulation (e.g., days, months).
    seed : int or None
        Random seed for reproducibility.
    """
    def __init__(self, n_sims: int = 10_000, horizon: int = 252, seed: int | None = 42):
        self.n_sims = n_sims
        self.horizon = horizon
        self.rng = np.random.default_rng(seed)

    # ---------- USER-PLUGGABLE MODEL ----------
    def evolve(self, state, t: int, params: dict):
        """
        Evolve system state one step.
        Override or modify this to match your process.
        Example below uses Geometric Brownian Motion (GBM) for prices.
        """
        mu  = params.get("mu", 0.05)         # drift
        sigma = params.get("sigma", 0.2)     # volatility
        dt = params.get("dt", 1/252)         # time step
        shock = self.rng.normal(0, np.sqrt(dt))
        return state * np.exp((mu - 0.5 * sigma**2) * dt + sigma * shock)

    def simulate_path(self, initial_state: float, params: dict):
        """
        Runs a single path and returns the states at each time step.
        """
        path = np.empty(self.horizon + 1)
        path[0] = initial_state
        for t in range(1, self.horizon + 1):
            path[t] = self.evolve(path[t-1], t, params)
        return path

    def run(self, initial_state: float, params: dict, kpi_fn=None):
        """
        Runs all simulations and returns:
            - DataFrame of KPIs (one row per simulation)
            - Optional raw states (if requested via kpi_fn)
        kpi_fn(state_path) -> dict
             Computes KPIs for a single path (e.g., final value, drawdown, VaR).
        """
        results = []
        for _ in range(self.n_sims):
            path = self.simulate_path(initial_state, params)
            if kpi_fn is None:
                # Default KPIs
                kpis = {
                    "final": path[-1],
                    "min": path.min(),
                    "max": path.max(),
                    "mean": path.mean(),
                }
            else:
                kpis = kpi_fn(path)
            results.append(kpis)

        df = pd.DataFrame(results)
        return df

# ---------- Example 1: Portfolio value & VaR ----------
def portfolio_kpis(path, confidence=0.95):
    # daily returns from path
    returns = np.diff(path) / path[:-1]
    final_value = path[-1]
    # one-day loss distribution (negative returns)
    losses = -returns
    var = np.quantile(losses, confidence)
    cvar = losses[losses >= var].mean() if np.any(losses >= var) else np.nan
    return {
        "final_value": final_value,
        "mean_return": returns.mean(),
        "std_return": returns.std(ddof=1),
        f"VaR_{int(confidence*100)}%": var,
        f"CVaR_{int(confidence*100)}%": cvar,
        "max_drawdown": (1 - path / np.maximum.accumulate(path)).max()
    }

if __name__ == "__main__":
    # Configure simulation
    sim = MonteCarloSimulator(n_sims=20_000, horizon=252, seed=7)
    initial_price = 100.0
    params = {"mu": 0.06, "sigma": 0.25, "dt": 1/252}

    # Run
    df = sim.run(initial_state=initial_price, params=params,
                 kpi_fn=lambda p: portfolio_kpis(p, confidence=0.99))

    # Summary stats
    summary = df.describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).T
    print("\n=== KPI summary across simulations ===")
    print(summary)

    # Probability of breaching thresholds
    prob_below_80 = (df["final_value"] < 80).mean()
    prob_above_120 = (df["final_value"] > 120).mean()
    print(f"\nP(final_value < 80): {prob_below_80:.3%}")
    print(f"P(final_value > 120): {prob_above_120:.3%}")

    # Export results if needed


=== KPI summary across simulations ===
                count        mean        std        min         1%         5%  \
final_value   20000.0  106.085779  26.703657  32.205950  57.535898  68.422699   
mean_return   20000.0    0.000236   0.000986  -0.004369  -0.002069  -0.001383   
std_return    20000.0    0.015738   0.000701   0.013137   0.014134   0.014589   
VaR_99%       20000.0    0.034793   0.003153   0.023546   0.028083   0.029840   
CVaR_99%      20000.0    0.039196   0.003914   0.026182   0.031042   0.033115   
max_drawdown  20000.0    0.241479   0.087575   0.067227   0.099114   0.123821   

                     50%         95%         99%         max  
final_value   102.956225  154.589564  183.063533  255.375181  
mean_return     0.000238    0.001857    0.002521    0.003853  
std_return      0.015729    0.016884    0.017377    0.018889  
VaR_99%         0.034668    0.040270    0.042841    0.049494  
CVaR_99%        0.039002    0.045996    0.049226    0.056785  
max_drawdown  